In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [3]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [4]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [ ]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[]


In [136]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        self.opinions = {}
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input}) (Opinions: {self.opinions})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [178]:
Sami = Agent("Sami", "agent_001", "Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.", "male")

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.", "female")

Sami.opinions["Eliana"] = "He likes Eliana and thinks her cooking advices are great"
Eliana.opinions["Sami"] = "She thinks Sami is cool and she likes his cooking"

print(Sami)
print(Eliana)

Agent Sami (ID: agent_001) (Description: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has a very bad and annoying personality. Other than that, he has an opinion on everything else.) (Opinions: {'Eliana': 'He likes Eliana and thinks her cooking advices are great'})
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is a close friend to Gordon Ramsay who taught her everything he knows about cooking. She is the sweetest person on earth and would never lie or attack someone.) (Opinions: {'Sami': 'She thinks Sami is cool and she likes his cooking'})


In [185]:
def make_initial_prompt(agent1, agent2, subject):
    message_content = f"""
            Context:
            Here is a description of {agent1.name}: {agent1.user_input}.
            Here is a description of {agent2.name}: {agent2.user_input}.
            Here is what {agent1.name} thinks about {agent2.name}:
            {agent1.opinions[agent2.name]}
            Here is what {agent2.name} thinks about {agent1.name}:
            {agent2.opinions[agent1.name]}
            The context of this conversation is: {subject}
            Start directly with the conversation, not a introduction phrase. Create the conversation they had:

            {agent1.name}: "
        """
    return message_content

def make_subject(memory):
    message_content = f"""
            Context:
            This is the memory of a conversation between 2 people:
            {memory}
            Make a summary in 1 sentence or 2 with the important points and details of the conversation (for example, if they are talking about doing an activity or a dish together, keep it in memory), ignorig the first sentence.
            Start directly with the summary, not with a phrase like "here is a summary".
        """
    return message_content

In [180]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, subject, use_memory = True):
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    if (len(memories) > 0 and use_memory):
        subject_prompt = make_subject(memories[0])
        data = {
            "model": "llama-3.2-1b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": subject_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }
        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        subject = "Here is the resume of the past conversation: " + response + " They meet some time later"
        print("Subject :", subject)
    
    message_prompt = make_initial_prompt(agent1, agent2, subject)

    data = {
            "model": "llama-3.2-1b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    temp_store_memory(agent1.agent_id, [agent2.agent_id], "Had this conversation with " + agent2.name + ": " + str(response))
    temp_store_memory(agent2.agent_id, [agent1.agent_id], "Had this conversation with " + agent1.name + ": " + str(response))

In [186]:
chat(Sami, Eliana, "Sami is cooking pasta")

Subject : Here is the resume of the past conversation: Sami and Eliana discussed their fishing trip, where Sami caught a huge fish and plans to make spaghetti with a rosemary-garlic sauce, while also joking about Gordon Ramsay's antics in the kitchen. They meet some time later
Sami: Oh great, Eliana is here. I was just thinking of making spaghetti and rosemary-garlic sauce for dinner tonight. You know, because that's exactly what Gordon Ramsay would make if he were here.

Eliana: (laughs) Ah, Sami, you're a genius! I've been meaning to ask you, how's the new recipe coming along? Is it going to be as epic as last time?

Sami: Epic? Are you kidding me? It'll probably just be a bunch of overcooked spaghetti with some mediocre sauce. But hey, that's what I need – a little bit of mediocrity in my life.

Eliana: (giggles) Oh Sami, you're so funny! And don't even get me started on Gordon Ramsay's latest antics. Have you seen the videos of him yelling at the contestants?

Sami: (chuckles) Yeah